
*   Computing Platforms: Set up the Workspace for Machine Learning Projects.  https://ms.pubpub.org/pub/computing
*  Machine Learning for Predictions. https://ms.pubpub.org/pub/ml-prediction
* Machine Learning Packages: https://scikit-learn.org/stable/


# Part I: Import and Inspect Data

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/Rising-Stars-by-Sunshine/stats201-Final-Project-Ace/main/data/Queried_Data/Remote%20Work%20Google%20Trends%20-%20Sheet1.csv')
df.head()

,Week,remote work: (United States)
0,3/4/18,28
1,3/11/18,34
2,3/18/18,32
3,3/25/18,32
4,4/1/18,30


# Part II: Prepare the Y varible for Regression

## 2.2. Make Sure that the Data Type of Y is "numeric"

In [3]:
df=df.rename(columns={"remote work: (United States)": "theta"})



In [4]:
df.dtypes
df.head()

,Week,theta
0,3/4/18,28
1,3/11/18,34
2,3/18/18,32
3,3/25/18,32
4,4/1/18,30


In [5]:
df['theta'] = pd.to_numeric(df['theta'])
df.dtypes

Week     object
theta     int64
dtype: object

# Part IV: Create the X variables

## 4.1. Shift the Y to get past values

reference:
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shift.html

In [6]:
# generate a new variable as the previous 1 observable of your Y variable for regression
df['theta_past'] =df['theta'].shift(1)
df.head()

,Week,theta,theta_past
0,3/4/18,28,NaN
1,3/11/18,34,28.0
2,3/18/18,32,34.0
3,3/25/18,32,32.0
4,4/1/18,30,32.0


## 4.2. Calculate the Moving Averages

references: 

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html

https://towardsdatascience.com/moving-averages-in-python-16170e20f6c

In [7]:
#@title Define the Window
window = 10 #@param {type:"number"}


In [8]:
df['theta_past_ma10']=df['theta_past'].rolling(window=window,min_periods=1).mean()
df.head(20)

,Week,theta,theta_past,theta_past_ma10
0,3/4/18,28,NaN,NaN
1,3/11/18,34,28.0,28.000000
2,3/18/18,32,34.0,31.000000
3,3/25/18,32,32.0,31.333333
4,4/1/18,30,32.0,31.500000
5,4/8/18,24,30.0,31.200000
6,4/15/18,31,24.0,30.000000
7,4/22/18,26,31.0,30.142857
8,4/29/18,30,26.0,29.625000
9,5/6/18,28,30.0,29.666667


# Part V Train and Test Split

*reference*:

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html

In [9]:
from sklearn.model_selection import TimeSeriesSplit
tss = TimeSeriesSplit()
print(tss)

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)


In [10]:
# change the train and test split parameters 
tss = TimeSeriesSplit(gap=0, max_train_size=None, n_splits=2, test_size=None)

In [11]:
for train_idx, test_idx in tss.split(df):
    print("TRAIN:", train_idx, "TEST:", test_idx)

TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87] TEST: [ 88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105
 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123
 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141
 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159
 160 161 162 163 164 165 166 167 168 169 170 171 172 173]
TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88 

In [12]:
train_idx

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173])

In [13]:
test_idx

array([174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186,
       187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199,
       200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212,
       213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225,
       226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238,
       239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251,
       252, 253, 254, 255, 256, 257, 258, 259])

In [14]:
train_df = df.filter(items=train_idx, axis=0)
test_df =  df.filter(items=test_idx, axis=0)

In [15]:
train_df.head()

,Week,theta,theta_past,theta_past_ma10
0,3/4/18,28,NaN,NaN
1,3/11/18,34,28.0,28.000000
2,3/18/18,32,34.0,31.000000
3,3/25/18,32,32.0,31.333333
4,4/1/18,30,32.0,31.500000


In [16]:
test_df.head()

,Week,theta,theta_past,theta_past_ma10
174,7/4/21,55,58.0,53.3
175,7/11/21,60,55.0,54.1
176,7/18/21,56,60.0,55.4
177,7/25/21,60,56.0,55.2
178,8/1/21,59,60.0,56.0


# Part VI Prepare the Train and Test Data for Classification and Regression

## 6.2 Regression

### 6.2.1. Define the columns (Y, X) for Regression

In [17]:
cols_R = ['theta','theta_past_ma10']

### 6.2.2. Define the Data Frame of Train and Test Data for Regression

In [18]:
df_R_train = train_df[cols_R]
df_R_test = test_df[cols_R]

### 6.2.3. Export the Train and Test Data for Regression

In [19]:
df_R_train.head()

,theta,theta_past_ma10
0,28,NaN
1,34,28.000000
2,32,31.000000
3,32,31.333333
4,30,31.500000


In [20]:
df_R_train.to_csv('Regression_Train.csv')

In [21]:
ls

Regression_Test.csv  Regression_Train.csv  sample_data/


In [22]:
df_R_test.head()

,theta,theta_past_ma10
174,55,53.3
175,60,54.1
176,56,55.4
177,60,55.2
178,59,56.0


In [23]:
df_R_test.to_csv('Regression_Test.csv')